In [3]:
from googletrans import Translator

In [4]:
translator = Translator()

In [84]:
print(translator.translate('Солнечный', src='ru', dest='en').extra_data['definitions'][0][1]) #, dest='en' #all-translations #possible-translations #definitions

[['Основанный на пользовании энергией солнца.', '12372153537602071154', 'Солнечные часы прибор для определения времени по солнцу'], ['Наполненный ярким светом солнца.', '2317829880244569869', 'С. день']]


In [75]:
obj = translator.translate('sunny', src='en', dest='ru').__dict__

In [76]:
print(obj['extra_data']['definitions'])

[['имя прилагательное', [['bright with sunlight.', 'm_en_gbus1013150.003', 'a sunny day']], 'sunny']]


In [92]:
print(obj['extra_data']['all-translations'][0])

['наречие', ['солнечно'], [['солнечно', ['sunny'], None, 0.34023288]], 'sunny', 4]


In [94]:
print(translator.translate('lord', src='en', dest='ru').extra_data['possible-translations'][0])

[['lord', None, [['господин', 976, True, False], ['повелитель', 21, True, False], ['лорд', 2, True, False], ['господина', 0, True, False], ['владыка', 0, True, False]], [[0, 4]], 'lord', 0, 1]]


In [7]:
import pandas as pd
import numpy as np
import re

In [23]:
lang_df = pd.DataFrame(columns=['origin', 'translation', 'all-translations', 'possible-translations', 'definitions'])

In [21]:
languages = ["en", "de", "fr", "es"]
terms_li = []
with open('pt.asc') as f:
    lines = f.read().strip('\n').split('\n')
    i=0
    for i, line in enumerate(lines):
        try:
            term = re.search('\$[^$]+\$' , line).group()[1:-1]
            terms_li.append(term)
        except:
            print(line)

In [ ]:
def get_reverse_translate(origin, lang, attr):
    if attr == 'definitions':
        definitions = getattr(translator.translate(origin, src='ru', dest=lang), 'extra_data')['definitions']
        if definitions is None:
            return '-'
        definitions = [i[0][1] for i in definitions]
        if len(definitions)>1:
            definitions = sum(definitions, [])
        definitions = [i[0] for i in definitions]
        return ' | '.join(definitions)
    translation = translator.translate(origin, src='ru', dest=lang).text
    translation_obj = translator.translate(translation, src=lang, dest='ru')
    if attr == 'translation':
        return translation_obj.text
    attr_value = getattr(translation_obj, 'extra_data')[attr]
    if attr_value is None:
            return '-'
    if attr == 'all-translations':
        all_translations = [i[1] for i in attr_value]
        all_translations = sum(all_translations, [])
        return ' | '.join(all_translations)
    elif attr == 'possible-translations':
        possible_translations = attr_value[0][2]
        return ' | '.join([i[0] for i in possible_translations])
            
for lang in languages:
    lang_df = pd.DataFrame(columns=['origin', 'translation', 'all-translations', 'possible-translations', 'definitions'])
    table_name = 'pt_' + lang + '.csv'
    for term in terms_li:
        row = {'origin': term, 'all-translations': None, 'possible-translations': None, 'definitions': None}
        for attr in lang_df.columns[1:]:
            row[attr] = get_reverse_translate(term, lang, attr)
        s = pd.Series(row)
        lang_df = lang_df.append(s, ignore_index=True)
    lang_df.to_csv(table_name)

In [24]:
for attr in lang_df.columns[1:]:
    print(attr)

all-translations
possible-translations
definitions


In [26]:
s = pd.Series({'origin': 'a', 'all-translations': 'b', 'possible-translations': 'c', 'definitions': 'd'})
lang_df = lang_df.append(s, ignore_index=True)
lang_df.head()

,origin,all-translations,possible-translations,definitions
0,a,b,c,d
